<a href="https://colab.research.google.com/github/AlbertoRoldanSastre/AlbertoRoldanSastre/blob/main/codeYOLOv8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Memoria ram y capacidad de memoria

In [1]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


Chequear si tengo gpu y cual

In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

/bin/bash: line 1: nvidia-smi: command not found


Importar imágenes y librerías

estas en rar, hay que descomprimir y eso

In [ ]:
#0. En Colab, primero instala los paquetes y monta tu Drive:  Instalar las librerías necesarias
!pip install ultralytics wandb  # Esto instalará ultralytics y wandb (torch y torchvision generalmente ya vienen instalados en Colab)

import ultralytics
ultralytics.checks()

# 1. Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')           #cambiar la ruta de drive de donde lo extraes

# 2. Instalar unrar para poder descomprimir archivos RAR
!apt-get update
!apt-get install -y unrar

# 3. Descomprimir el archivo RAR desde Google Drive
# Ajusta la ruta del archivo RAR a la ubicación correcta en tu Drive.
# Este comando extrae el contenido del archivo a la carpeta /content/dataset/
#!unrar x "/content/drive/MyDrive/YourDatasetFolder/your_dataset.rar" /content/dataset/


#OPCION A:

# 4. Definir las rutas de los archivos RAR en Drive
# Ajusta estas rutas según dónde estén ubicados en tu Drive
images_train_rar = "/content/drive/MyDrive/model_rar_640_tiles_labels/train/images.rar"
labels_train_rar = "/content/drive/MyDrive/model_rar_640_tiles_labels/train/labels.rar"
images_validation_rar = "/content/drive/MyDrive/model_rar_640_tiles_labels/valid/images.rar"
labels_validation_rar = "/content/drive/MyDrive/model_rar_640_tiles_labels//valid/labels.rar"

# 5. Definir las carpetas de destino en Colab para cada uno
# Estas carpetas mantendrán la estructura de YOLO (por ejemplo, "images" y "labels")
dest_train_images = "/content/dataset/train"    # he dejado image sy labels en el mismo directorio a ver si así se apaña el no detectar labels
dest_train_labels = "/content/dataset/train"
dest_val_images = "/content/dataset/val"
dest_val_labels = "/content/dataset/val"

import os
os.makedirs(dest_train_images, exist_ok=True)
os.makedirs(dest_train_labels, exist_ok=True)
os.makedirs(dest_val_images, exist_ok=True)
os.makedirs(dest_val_labels, exist_ok=True)

# 6. Descomprimir cada archivo RAR en su carpeta correspondiente
!unrar x "{images_train_rar}" "{dest_train_images}/"
!unrar x "{labels_train_rar}" "{dest_train_labels}/"
!unrar x "{images_validation_rar}" "{dest_val_images}/"
!unrar x "{labels_validation_rar}" "{dest_val_labels}/"

# Opcional: listar el contenido para confirmar que se descomprimió correctamente
!ls /content/dataset/train/       # he dejado image sy labels en el mismo directorio a ver si así se apaña el no detectar labels
!ls /content/dataset/train/
!ls /content/dataset/val/
!ls /content/dataset/val/


#OPCION B:

# 4. Iterar sobre cada archivo RAR en la carpeta de origen y descomprimirlo
#for file in os.listdir(source_folder):
#    if file.lower().endswith(".rar"):
#        rar_path = os.path.join(source_folder, file)
#        print("Descomprimiendo:", rar_path)
#        # Ejecuta el comando unrar para extraer el contenido al destination_folder
#        !unrar x "{rar_path}" "{destination_folder}/"


OVERSAMPLING: (solo con las fotos con etiquetas)
 (aumentar el número de fotos positivas solo para poder tener más que enseñar)

---



In [ ]:
import os
import shutil

# Rutas de las carpetas originales
orig_images_dir = "/content/dataset/train/images"
orig_labels_dir = "/content/dataset/train/labels"

# Carpeta de destino para el dataset balanceado
balanced_dir = "/content/balanced_train"
balanced_images_dir = os.path.join(balanced_dir, "images")
balanced_labels_dir = os.path.join(balanced_dir, "labels")

os.makedirs(balanced_images_dir, exist_ok=True)
os.makedirs(balanced_labels_dir, exist_ok=True)

################################################################
################################################################
# Factor de duplicación para imágenes positivas
dup_factor = 3                                                      ########cuantas veces las quiero multiplicar

#################################################################

# Recorremos todas las imágenes del dataset original
for filename in os.listdir(orig_images_dir):
    if not filename.lower().endswith(('.jpg','.jpeg','.png')):
        continue
    # Ruta de la imagen y de la etiqueta correspondiente
    orig_img_path = os.path.join(orig_images_dir, filename)
    label_filename = os.path.splitext(filename)[0] + ".txt"
    orig_label_path = os.path.join(orig_labels_dir, label_filename)

    # Determinar si la imagen es positiva (tiene anotaciones) o negativa
    is_positive = False
    if os.path.exists(orig_label_path):
        with open(orig_label_path, 'r') as f:
            content = f.read().strip()
        if content:  # Si hay algo, es positiva
            is_positive = True

    # Copiar la imagen y el archivo de etiqueta al nuevo directorio
    # Si es positiva, duplicamos según el factor
    copies = dup_factor if is_positive else 1
    for i in range(copies):
        # Para evitar sobreescritura, agregamos un sufijo en el nombre de la imagen duplicada
        new_filename = os.path.splitext(filename)[0]
        if copies > 1:
            new_filename += f"_{i+1}"
        new_filename += os.path.splitext(filename)[1]
        new_img_path = os.path.join(balanced_images_dir, new_filename)
        shutil.copy2(orig_img_path, new_img_path)

        # También copiamos la etiqueta (si existe), ajustando el nombre
        if os.path.exists(orig_label_path):
            new_label_filename = os.path.splitext(label_filename)[0]
            if copies > 1:
                new_label_filename += f"_{i+1}"
            new_label_filename += ".txt"
            new_label_path = os.path.join(balanced_labels_dir, new_label_filename)
            shutil.copy2(orig_label_path, new_label_path)

print("Dataset balanceado creado en:", balanced_dir)


**DATA AUGMENTATION**
solo aplicado a las fotos previamente procesadas (con etiquetas)

produce una imagen por cada transformacion mas la original mas labels que contienen la info de las clases

In [ ]:
import os
import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2
import shutil

# Función para leer anotaciones en formato YOLO
def read_yolo_labels(label_path):
    """
    Lee un archivo de etiquetas YOLO y devuelve dos listas:
    - bboxes: Lista de bounding boxes en formato [x_center, y_center, width, height] (valores normalizados)
    - classes: Lista de clases (como string o número) correspondiente a cada bbox.
    """
    bboxes = []
    classes = []
    if os.path.exists(label_path):
        with open(label_path, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) == 5:
                    classes.append(parts[0])
                    bbox = list(map(float, parts[1:]))
                    bboxes.append(bbox)
    return bboxes, classes

# Definir pipelines individuales para cada transformación (se aplican siempre, p=1.0)
pipeline_flip = A.Compose([
    A.HorizontalFlip(p=1.0),
    A.Resize(640, 640),
    ToTensorV2()
], bbox_params=A.BboxParams(format='yolo', label_fields=['labels']))

pipeline_rotate = A.Compose([
    A.Rotate(limit=15, p=1.0),  # Rota entre -15° y +15°
    A.Resize(640, 640),
    ToTensorV2()
], bbox_params=A.BboxParams(format='yolo', label_fields=['labels']))

pipeline_shear = A.Compose([
    A.Affine(shear=15, p=1.0),
    A.Resize(640, 640),
    ToTensorV2()
], bbox_params=A.BboxParams(format='yolo', label_fields=['labels']))

pipeline_bright_contrast = A.Compose([
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=1.0),
    A.Resize(640, 640),
    ToTensorV2()
], bbox_params=A.BboxParams(format='yolo', label_fields=['labels']))

pipeline_blur = A.Compose([
    A.Blur(blur_limit=7, p=1.0),
    A.Resize(640, 640),
    ToTensorV2()
], bbox_params=A.BboxParams(format='yolo', label_fields=['labels']))

pipeline_noise = A.Compose([
    A.GaussianNoise(var_limit=(10.0, 50.0), p=1.0),
    A.Resize(640, 640),
    ToTensorV2()
], bbox_params=A.BboxParams(format='yolo', label_fields=['labels']))

# Pipeline sin augmentation (solo resize) para referencia
pipeline_no_aug = A.Compose([
    A.Resize(640, 640),
    ToTensorV2()
], bbox_params=A.BboxParams(format='yolo', label_fields=['labels']))

# Lista de pipelines con un sufijo para el nombre de archivo
pipelines = [
    ("flip", pipeline_flip),
    ("rotate", pipeline_rotate),
    ("shear", pipeline_shear),
    ("bright", pipeline_bright_contrast),
    ("blur", pipeline_blur),
    ("noise", pipeline_noise),
    ("noaug", pipeline_no_aug)
]

def apply_pipeline(image_path, bboxes, labels, pipeline):
    """
    Aplica un pipeline de augmentación a una imagen y sus bounding boxes.
    Retorna la imagen aumentada (tensor convertido a numpy) y las nuevas bounding boxes.
    """
    image = cv2.imread(image_path)
    if image is None:
        return None, None
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    augmented = pipeline(image=image, bboxes=bboxes, labels=labels)
    aug_img = augmented["image"]  # Tensor (C, H, W) con valores en [0,1]
    # Convertir a numpy (H, W, C), escalar a 0-255 y volver a BGR para guardar con cv2
    aug_img_np = aug_img.permute(1, 2, 0).cpu().numpy() * 255
    aug_img_np = aug_img_np.astype('uint8')
    aug_img_np = cv2.cvtColor(aug_img_np, cv2.COLOR_RGB2BGR)
    return aug_img_np, augmented["bboxes"]

# Directorios de entrada (dataset original)
input_images_dir = "/content/dataset/train/images"
input_labels_dir = "/content/dataset/train/labels"

# Directorios de salida para el dataset aumentado
output_images_dir = "/content/dataset_aug/train/images"
output_labels_dir = "/content/dataset_aug/train/labels"

os.makedirs(output_images_dir, exist_ok=True)
os.makedirs(output_labels_dir, exist_ok=True)

# Iterar sobre cada imagen en el dataset de entrenamiento
for filename in os.listdir(input_images_dir):
    if not filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        continue
    img_path = os.path.join(input_images_dir, filename)
    label_filename = os.path.splitext(filename)[0] + ".txt"
    label_path = os.path.join(input_labels_dir, label_filename)

    # Leer anotaciones
    bboxes, classes = read_yolo_labels(label_path)

    # Si no hay anotaciones, decide si deseas aplicar augmentación o solo copiar la imagen
    if len(bboxes) == 0:
        # Por ejemplo, copia la imagen original sin cambios
        shutil.copy2(img_path, os.path.join(output_images_dir, filename))
        # Crea un archivo de etiquetas vacío en el output
        open(os.path.join(output_labels_dir, label_filename), 'w').close()
        continue

    # Aplicar cada pipeline a la imagen positiva
    for suffix, pipeline in pipelines:
        aug_img, aug_bboxes = apply_pipeline(img_path, bboxes, classes, pipeline)
        if aug_img is None:
            continue
        # Genera un nuevo nombre de archivo que incluya el sufijo de la transformación
        new_img_name = os.path.splitext(filename)[0] + f"_{suffix}" + os.path.splitext(filename)[1]
        new_label_name = os.path.splitext(filename)[0] + f"_{suffix}" + ".txt"
        # Guardar la imagen aumentada
        cv2.imwrite(os.path.join(output_images_dir, new_img_name), aug_img)
        # Guardar las anotaciones aumentadas en el mismo formato YOLO
        with open(os.path.join(output_labels_dir, new_label_name), 'w') as f:
            for cls, bbox in zip(classes, aug_bboxes):
                f.write(f"{cls} {bbox[0]:.6f} {bbox[1]:.6f} {bbox[2]:.6f} {bbox[3]:.6f}\n")
    print(f"Procesada {filename} con todos los pipelines.")

print("Data augmentation completo: se han generado múltiples imágenes por cada imagen original.")


WANDB

In [ ]:

# 8. Autenticación en Weights & Biases (W&B)
import wandb
from wandb.integration.ultralytics import add_wandb_callback

# Initialize your Weights & Biases environment
wandb.login(key="105f813f52ef215433a521087f025ed76bf34c23")  # Te pedirá ingresar tu API key o autenticarte desde el navegador.

# Enable Weights & Biases logging
yolo_settings_wandb=True

# Disable Weights & Biases logging
#yolo settings wandb=False


#8.1 Initialize a Weight and Biases run:
wandb.init(project= "seals_2025 ", job_type= "training")








mejorar gpu computación

In [3]:
# 5. Configurar PyTorch para GPU (opcional: optimización extra)
import torch
if torch.cuda.is_available():
    torch.backends.cudnn.benchmark = True  # mejora la velocidad en convoluciones si el input es fijo

YOLO

In [ ]:

# 9. Ejecutar el entrenamiento de YOLOv8
from ultralytics import YOLO

# Carga el modelo (en este ejemplo, YOLOv8s)
model = YOLO("yolov8s.pt")

#Add W&B callback for ultralytics:
add_wandb_callback(model, True, True)

# Ejecuta el entrenamiento utilizando el archivo data.yaml en la carpeta extraída.
model.train(
    project="seals_2025",
    name="focas_training_prueba1_640_24_02_25",
    data="/content/drive/MyDrive/model_rar_640_tiles_labels/data.yaml",  # Asegúrate de que data.yaml use rutas correctas para Colab
    epochs=5,
    imgsz=640,
    batch=16,
    lr0=0.001,
    optimizer="adamw",
    augment=False,      # Desactiva el data augmentation on the fly de YOLO##############  asi puedo hacer yo la data augmentation con las fotos con labels
    half=True,           # Activa precisión mixta (FP16) para GPU
    workers=16,          # Número de workers para el DataLoader
    device='cuda'        # Fuerza el uso de GPU
)





Gráficas durante aprendizaje en Tensorboard

In [ ]:
%load_ext tensorboard

# Iniciar TensorBoard apuntando al directorio donde se guardan los logs
%tensorboard --logdir /content/seals_2025/focas_training_prueba1_640_24_02_25


Guardar resultados

In [ ]:
# 10. Guardar los resultados del experimento

# Define la carpeta de destino en Google Drive
dest_folder = "/content/drive/MyDrive/model_rar_640_tiles_labels/foquitas"
!mkdir -p "{dest_folder}"                                                           #El comando mkdir -p crea esa carpeta (y cualquier subcarpeta necesaria) si no existe.

# Copia la carpeta de resultados (ajusta el nombre según el que se haya generado)
!cp -r /content/seals_2025/focas_training_prueba1_640_24_02_25 "{dest_folder}/"   #copiar los reultados del modelo en drive


# Comprueba que se copiaron los archivos (opcional)
!ls "{dest_folder}"                                                                 #verifica que los archivos se han copiado correctamente

# Opcional: comprimir la carpeta de resultados para descargarla a tu PC
!zip -r /content/foquitas.zip "{dest_folder}/focas_training_prueba1_640_24_02_25"   #Aquí se comprime la carpeta que acabas de copiar en Drive (la subcarpeta focas_training_prueba1_640_24_02_25 dentro de foquitas) en un archivo ZIP llamado foquitas.zip que se guarda en /content (la carpeta principal del entorno de Colab).

# Descargar el archivo ZIP a tu PC
from google.colab import files
files.download('/content/foquitas.zip')                                             #Este comando abre una ventana de descarga para que puedas bajar el archivo ZIP a tu ordenador.